# Blah

In [1]:
from IPython.display import display, HTML, Image , Markdown
from snowflake.snowpark.session import Session
from snowflake.snowpark.types import * 
import snowflake.snowpark.functions as F
import os ,configparser ,json

# Source various helper functions
%run ../scripts/notebook_helpers.py

PROJECT_HOME_DIR = '../../'
CONFIG_FL = f'{PROJECT_HOME_DIR}/config.ini'

In [2]:
display(Markdown("### Initialization"))

config = configparser.ConfigParser()
sp_session = None

print(" Initialize Snowpark session")
with open(CONFIG_FL) as f:
    config.read(CONFIG_FL)
    snow_conn_flpath =  f"{PROJECT_HOME_DIR}/{config['APP']['connection_fl']}"
    
    # ------------
    # Connect to snowflake
    with open(snow_conn_flpath) as conn_f:
        snow_conn_info = json.load(conn_f)
        sp_session = Session.builder.configs(snow_conn_info).create()

if(sp_session == None):
    raise(f'Unable to connect to snowflake. Validate connection information in file: {CONFIG_FL} ')

sp_session.sql(f''' use role {config['APP']['role']}; ''').collect()
sp_session.sql(f''' use database {config['APP']['database']}; ''').collect()
sp_session.sql(f''' use warehouse {config['APP']['warehouse']}; ''').collect()

df = sp_session.sql('select current_account() ,current_user() ,current_role();').to_pandas()
display(df)

### Initialization

 Initialize Snowpark session


,CURRENT_ACCOUNT(),CURRENT_USER(),CURRENT_ROLE()
0,ANA95816,VSEKAR,DEV_PCTRANSPERANCY_DEMO_RL


In [3]:
from datetime import datetime
from datetime import timedelta
import time
import hashlib

data_file = '2022_10_01_priority_health_HMO_in-network-rates.zip'
stage_path = 'data_stg/price_transperancy'

# data_file = '2022_07_01_priority_health_HMO_in-network-rates.json'
# data_file = '2022_07_01_priority_health_HMO_in-network-rates.json'
# data_file = '2022-11-01_cigna-health-life-insurance-company_national-oap_in-network-rates.json.gz'
# stage_path = 'ext_data_stg/data'

data_file_hash = hashlib.md5(data_file.encode()).hexdigest()
data_file_hash

'092c884b20ce621689355dd8dec5743e'

In [4]:
# Upload libraries to stage
lib_stage = config['APP']['lib_stage']

upload_locallibraries_to_p_stage(sp_session ,'../python/in-network-rates_v2' ,config['APP']['database'] ,'public' ,lib_stage ,'scripts')

print(' List stage directory !!')
stage_spdf = sp_session.sql(f'''list @{config['APP']['database']}.public.{lib_stage}; ''').collect()

for r in stage_spdf:
    print(f'@{r["name"]}')


 Uploading library to stage: sflk_pricing_transperancy.public.lib_stg 
    ../python/in-network-rates_v2/ingest-in-network-rates-seg-header_sp_v2.py => @lib_stg/scripts
    ../python/in-network-rates_v2/in-network-rates-segment-dagbuilder-v2-segment-count.py => @lib_stg/scripts
    ../python/in-network-rates_v2/ingest-in-network-rates-segment-lists_sp.py => @lib_stg/scripts
    ../python/in-network-rates_v2/in-network-rates-segment-dagbuilder-v2.py => @lib_stg/scripts
 List stage directory !!
@lib_stg/scripts/in-network-rates-segment-dagbuilder-v2-segment-count.py
@lib_stg/scripts/in-network-rates-segment-dagbuilder-v2.py
@lib_stg/scripts/ingest-in-network-rates-seg-header_sp_v2.py
@lib_stg/scripts/ingest-in-network-rates-segment-lists_sp.py


In [5]:
# Negotiation header parsing

start_time = time.time()
print(f'Started at: {datetime.now().strftime("%H:%M:%S")}')

sql_stmts = [
    f''' alter warehouse dev_pctransperancy_demo_wh set warehouse_size = XXLARGE; '''
    ,f'''truncate table {config['APP']['database']}.public.in_network_rates_segment_header_V2; '''
    ,f''' call {config['APP']['database']}.public.innetwork_rates_segheader(
            1000000 ,'{stage_path}' ,'{data_file}');'''
]
for stmt in sql_stmts:
    print(stmt)
    sp_session.sql(stmt).collect()

end_time = time.time()
print(f'Ended at: {datetime.now().strftime("%H:%M:%S")}')

elapsed_time = end_time - start_time
elapsed = str(timedelta(seconds=elapsed_time))
print(f'Elapsed: {elapsed}')


Started at: 08:21:37
 alter warehouse dev_pctransperancy_demo_wh set warehouse_size = XXLARGE; 
truncate table sflk_pricing_transperancy.public.in_network_rates_segment_header_V2; 
 call sflk_pricing_transperancy.public.innetwork_rates_segheader(
            1000000 ,'data_stg/price_transperancy' ,'2022_10_01_priority_health_HMO_in-network-rates.zip');
Ended at: 08:39:14
Elapsed: 0:17:37.109181


In [6]:
# Dag & task build up

start_time = time.time()
print(f'Started at: {datetime.now().strftime("%H:%M:%S")}')

sql_stmts = [
    f''' alter warehouse dev_pctransperancy_demo_wh set warehouse_size = XSMALL; '''
    ,f'''truncate table {config['APP']['database']}.public.task_to_segmentids; '''
    ,f'''truncate table {config['APP']['database']}.public.negotiated_arrangment_segments_v2; '''
    ,f'''truncate table {config['APP']['database']}.public.segment_task_execution_status; '''
    ,f''' call {config['APP']['database']}.public.innetwork_rates_dagbuilder_v2(
        100000 ,'{stage_path}' ,'{data_file}'
        ,20 ,current_warehouse() ,true
        );'''
]
for stmt in sql_stmts:
    print(stmt)
    sp_session.sql(stmt).collect()

end_time = time.time()
print(f'Ended at: {datetime.now().strftime("%H:%M:%S")}')

elapsed_time = end_time - start_time
elapsed = str(timedelta(seconds=elapsed_time))
print(f'Elapsed: {elapsed}')


Started at: 08:39:14
 alter warehouse dev_pctransperancy_demo_wh set warehouse_size = XSMALL; 
truncate table sflk_pricing_transperancy.public.task_to_segmentids; 
truncate table sflk_pricing_transperancy.public.negotiated_arrangment_segments_v2; 
truncate table sflk_pricing_transperancy.public.segment_task_execution_status; 
 call sflk_pricing_transperancy.public.innetwork_rates_dagbuilder_v2(
        100000 ,'data_stg/price_transperancy' ,'2022_10_01_priority_health_HMO_in-network-rates.zip'
        ,20 ,current_warehouse() ,true
        );
Ended at: 08:39:37
Elapsed: 0:00:22.796507


In [7]:
# segment parsing

start_time = time.time()
print(f'Started at: {datetime.now().strftime("%H:%M:%S")}')
dag_start_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
# XSMALL | SMALL | MEDIUM | LARGE | XLARGE | XXLARGE | XXXLARGE | X4LARGE | X5LARGE | X6LARGE
sql_stmts = [
    f''' alter warehouse dev_pctransperancy_demo_wh set max_concurrency_level = 3 '''
    ,f''' alter warehouse dev_pctransperancy_demo_wh set warehouse_size = X4LARGE; '''
    ,f'''truncate table {config['APP']['database']}.public.negotiated_arrangment_segments_v2; '''
    ,f'''truncate table {config['APP']['database']}.public.segment_task_execution_status; '''
    # ,f''' call {config['APP']['database']}.public.innetwork_rates_segments_ingest_sp(
    #     10000000 ,'{stage_path}' ,'{data_file}'
    #     ,'negotiated_rates' 
    #     ,'TSK_092C884B20CE621689355DD8DEC5743E_16');'''

    ,f''' execute task DAG_ROOT_{data_file_hash}; '''
]
for stmt in sql_stmts:
    sp_session.sql(stmt).collect()

end_time = time.time()
print(f'Ended at: {datetime.now().strftime("%H:%M:%S")}')

elapsed_time = end_time - start_time
elapsed = str(timedelta(seconds=elapsed_time))
print(f'Elapsed: {elapsed}')



Started at: 08:39:37


Failed to execute query [queryID: 01a8b333-0402-3d7c-0057-3c0301417d0e]  alter warehouse dev_pctransperancy_demo_wh set warehouse_size = XXXXLARGE; 
001421 (22023): SQL compilation error:
invalid type of property 'XXXXLARGE' for 'WAREHOUSE_SIZE'


SnowparkSQLException: (1304): 01a8b333-0402-3d7c-0057-3c0301417d0e: 001421 (22023): SQL compilation error:
invalid type of property 'XXXXLARGE' for 'WAREHOUSE_SIZE'

---

Observations

In [ ]:
# Monitor task to ensure that there are no errors

dag_start_time = '2022-11-25 10:11:00'
print(f'DAG start time: {dag_start_time}')

sql_stmt = f'''
select 
    name ,state ,error_code ,error_message
  from table(information_schema.task_history(
    scheduled_time_range_start=>to_timestamp_ltz('{dag_start_time} -0500')
    ,ERROR_ONLY => TRUE
  ))
  where state not in ('SUCCEEDED')
  order by state asc
;'''
df = sp_session.sql(sql_stmt).to_pandas()
display(df)


In [ ]:
sql_stmt = f'''
select min(l.inserted_at) as started_at ,max(l.inserted_at) as latest 
 ,TIMESTAMPDIFF(minutes, started_at ,latest) as elapsed_min_latest
 ,TIMESTAMPDIFF(minutes, started_at ,current_timestamp()) as elapsed_min 
 ,least(elapsed_min_latest ,elapsed_min)/60 as elapsed_hr
 ,count(l.task_name) as task_count
-- #,timediff(minutes, started_at ,latest) as elapsed_minutes 
from {config['APP']['database']}.public.segment_task_execution_status as l
 ;
    '''
df = sp_session.sql(sql_stmt).to_pandas()
display(df)

In [ ]:


sql_stmt = f'''
        select 'total_headers' ,sum(negotiated_rates_count)
            from {config['APP']['database']}.public.in_network_rates_segment_header_V2
        union
        select 'current_records_loaded' ,count(*) as rec_count 
            from {config['APP']['database']}.public.negotiated_arrangment_segments_v2
            group by data_file
        ;
    '''
df = sp_session.sql(sql_stmt).to_pandas()
display(df)

In [ ]:
sql_stmt = f'''
        select segment_id ,count(*) as rec_count 
        from {config['APP']['database']}.public.negotiated_arrangment_segments_v2
        group by segment_id
        ;
    '''
df = sp_session.sql(sql_stmt).to_pandas()
display(df)


In [ ]:
sql_stmt = f'''
        select *
        from {config['APP']['database']}.public.segment_task_execution_status
        ;
    '''
df = sp_session.sql(sql_stmt).to_pandas()
display(df)

In [ ]:
# x_df = df.set_index('SEGMENT_ID')
y_df = pd.qcut(df['NEGOTIATED_RATES_COUNT'], 10 ,labels=False)
# q_df.columns = 'CLASS'
# q_df

q_df = pd.DataFrame({'CLASS':y_df.values})
q_df
# q_df.value_counts()

In [ ]:
# df.join(q_df)
# q_df.columns
# df.columns
x_df = pd.concat([df ,q_df] ,axis=1)
x_df

In [ ]:
x_df.groupby('CLASS')['NEGOTIATED_RATES_COUNT'].sum()

In [ ]:
# from datetime import datetime
# now = datetime.now()

# start_time = now.strftime("%H:%M:%S")
# print(f'Started at: {start_time}')

# sql_stmts = [
#     f''' alter warehouse dev_pctransperancy_demo_wh set warehouse_size = XSMALL; '''
#     ,f'''truncate table {config['APP']['database']}.public.in_network_rates_segment_header_V2; '''
#     ,f''' call {config['APP']['database']}.public.innetwork_rates_segheader(
#             1000 ,'data_stg/price_transperancy' ,'2022_10_01_priority_health_HMO_in-network-rates.zip');'''
# ]
# for stmt in sql_stmts:
#     sp_session.sql(stmt).collect()

# now = datetime.now()
# end_time = now.strftime("%H:%M:%S")
# print(f'Ended at: {end_time}')

---
### TBL

In [ ]:
df = sp_session.table(f'''{config['APP']['database']}.public.in_network_rates_segment_header_V2''').limit(10).to_pandas()
display(df)

#### TODO : Perform some model analysis 
 - accuracy score
 - error rate calculation
 - confusion matrix
  ...

--- 
### Closeout

    With that we are finished this section of the demo setup

In [ ]:
# sp_session.close()
print('Finished!!!')

In [ ]:
employees = [('Stuti', 28, 'Varanasi', 20000),
            ('Saumya', 32, 'Delhi', 25000),
            ('Aaditya', 25, 'Mumbai', 40000),
            ('Saumya', 32, 'Delhi', 35000),
            ('Saumya', 32, 'Delhi', 30000),
            ('Saumya', 32, 'Mumbai', 20000),
            ('Aaditya', 40, 'Dehradun', 24000),
            ('Seema', 32, 'Delhi', 70000)
            ]
 
# Create a DataFrame object from list
df = pd.DataFrame(employees,
                columns =['Name', 'Age',
                        'City', 'Salary'])
# Show the dataframe
df

In [ ]:
df.iloc[3:5,]